In [1]:
# Logger
import logging
logging.basicConfig(level=logging.WARN)

# Enable automatic module reload
%load_ext autoreload
%autoreload 2

# Load PyALCS module
from lcs.agents.acs2 import ACS2, Configuration, ClassifiersList

# Load environments
import gym
import my_mazes

# Allow to parallelize all benchmarks to do
import parmap

# For calculation standard deviation
import statistics

In [2]:
# Utils section

from lcs.metrics import population_metrics
from lcs.strategies.subsumption import does_subsume

def _maze_knowledge(population, environment) -> float:
    transitions = environment.env.get_all_possible_transitions()
    # Take into consideration only reliable classifiers
    reliable_classifiers = [c for c in population if c.is_reliable()]
    # Count how many transitions are anticipated correctly
    nr_correct = 0
    # For all possible destinations from each path cell
    for start, action, end in transitions:
        p0 = environment.env.maze.perception(*start)
        p1 = environment.env.maze.perception(*end)
        if any([True for cl in reliable_classifiers
                if cl.predicts_successfully(p0, action, p1)]):
            nr_correct += 1
    return nr_correct / len(transitions) * 100.0

def _maze_metrics(pop, env):
    metrics = {
        'knowledge': _maze_knowledge(pop, env)
    }
    # Add basic population metrics
    metrics.update(population_metrics(pop, env))
    return metrics

def clean_population(agent, does_anticipate_change:bool = True):
        compact_pop = []
        for cl in agent.population:
            to_keep = True
            for other in agent.population:
                if does_subsume(other, cl, agent.cfg.theta_exp):
                    to_keep = False
                    break
            if to_keep:
                compact_pop.append(cl)
        agent.population = ClassifiersList(*compact_pop)

        if does_anticipate_change:
            pop = [cl for cl in agent.population if cl.does_anticipate_change()]
            agent.population = ClassifiersList(*pop)

# Agent - BACS - BENCHMARKING

## Main Parameters

In [3]:
CLASSIFIER_LENGTH = 8
NUMBER_OF_POSSIBLE_ACTIONS = 8

NUMBER_OF_EXPLORE_TRIALS = 1000
METRICS_TRIAL_FREQUENCY_EXPLORE = 20
BETA_EXPLORE = 0.05
EPSILON = 0.8

NUMBER_OF_EXPLOIT_TRIALS_NO_RL = 500
BETA_EXPLOIT_NO_RL = 0.00

NUMBER_OF_EXPLOIT_TRIALS_RL_START = 100
BETA_EXPLOIT_RL_START = 0.05

NUMBER_OF_EXPLOIT_TRIALS_RL = 500
BETA_EXPLOIT_RL = 0.05

NUMBER_OF_ITERATIONS_TO_BENCH = 30

NB_OF_PROCESSES = 24

## Full Details of Agent Parameters

In [4]:
cfg_explore = Configuration(
    classifier_length=CLASSIFIER_LENGTH,
    number_of_possible_actions=NUMBER_OF_POSSIBLE_ACTIONS,
    user_metrics_collector_fcn=_maze_metrics,
    metrics_trial_frequency=METRICS_TRIAL_FREQUENCY_EXPLORE,
    do_ga=False,
    do_subsumption=True,
    do_action_planning=False,
    beta=BETA_EXPLORE,
    gamma=0.95,
    theta_i=0.1,
    theta_r=0.9,
    epsilon=EPSILON,
    u_max=CLASSIFIER_LENGTH,
    theta_exp=20
)

cfg_exploit_no_rl = Configuration(
    classifier_length=CLASSIFIER_LENGTH,
    number_of_possible_actions=NUMBER_OF_POSSIBLE_ACTIONS,
    user_metrics_collector_fcn=_maze_metrics,
    metrics_trial_frequency=1,
    do_ga=False,
    do_subsumption=True,
    do_action_planning=False,
    beta=BETA_EXPLOIT_NO_RL,
    gamma=0.95,
    theta_i=0.1,
    theta_r=0.9,
    epsilon=0.0,
    u_max=CLASSIFIER_LENGTH,
    theta_exp=20
)

cfg_exploit_rl_start = Configuration(
    classifier_length=CLASSIFIER_LENGTH,
    number_of_possible_actions=NUMBER_OF_POSSIBLE_ACTIONS,
    user_metrics_collector_fcn=_maze_metrics,
    metrics_trial_frequency=1,
    do_ga=False,
    do_subsumption=True,
    do_action_planning=False,
    beta=BETA_EXPLOIT_RL_START,
    gamma=0.95,
    theta_i=0.1,
    theta_r=0.9,
    epsilon=0.0,
    u_max=CLASSIFIER_LENGTH,
    theta_exp=20
)

cfg_exploit_rl = Configuration(
    classifier_length=CLASSIFIER_LENGTH,
    number_of_possible_actions=NUMBER_OF_POSSIBLE_ACTIONS,
    user_metrics_collector_fcn=_maze_metrics,
    metrics_trial_frequency=1,
    do_ga=False,
    do_subsumption=True,
    do_action_planning=False,
    beta=BETA_EXPLOIT_RL,
    gamma=0.95,
    theta_i=0.1,
    theta_r=0.9,
    epsilon=0.0,
    u_max=CLASSIFIER_LENGTH,
    theta_exp=20
)

### Benchmarking - Maze

In [5]:
def acs2_bench_on_maze(env,n):
    avg_explore_list = []
    avg_exploit_no_rl_list = []
    avg_exploit_rl_start_list = []
    avg_exploit_rl_list = []
    knowledge_list = []
    population_list = []
    numerosity_list = []
    reliable_list = []
    
    for i in range(n):
        
        # Initialize environment
        maze = gym.make(env.id)
        
        # Reset it, by putting an agent into random position
        situation = maze.reset()
    
        # Training of ACS2 - Exploration
        agent_explore = ACS2(cfg_explore)
        population_explore, metrics_explore = agent_explore.explore(maze, NUMBER_OF_EXPLORE_TRIALS)
    
        # Using ACS2 - Exploitation - No RL module
        agent_exploit_no_rl = ACS2(cfg_exploit_no_rl, population_explore)
        clean_population(agent_exploit_no_rl, True)
        population_exploit_no_rl, metrics_exploit_no_rl = agent_exploit_no_rl.exploit(maze, NUMBER_OF_EXPLOIT_TRIALS_NO_RL)
        
        # Using ACS2 - Exploitation - Starting using RL module
        agent_exploit_rl_start = ACS2(cfg_exploit_rl_start, population_exploit_no_rl)
        clean_population(agent_exploit_rl_start, True)
        population_exploit_rl_start, metrics_exploit_rl_start = agent_exploit_rl_start.exploit(maze, NUMBER_OF_EXPLOIT_TRIALS_RL_START)
        
        # Using ACS2 - Exploitation - Using RL module
        agent_exploit_rl = ACS2(cfg_exploit_rl, population_exploit_rl_start)
        clean_population(agent_exploit_rl, True)
        population_exploit_rl, metrics_exploit_rl = agent_exploit_rl.exploit(maze, NUMBER_OF_EXPLOIT_TRIALS_RL)
        
        # Get frequencies to compute averages
        metrics_trial_frequency_explore = METRICS_TRIAL_FREQUENCY_EXPLORE
        metrics_trial_frequency_exploit = 1
        
        # Get average in exploration mode
        avg_step_explore = 0
        for trial in metrics_explore:
            avg_step_explore += trial['steps_in_trial']
        avg_step_explore /= NUMBER_OF_EXPLORE_TRIALS / metrics_trial_frequency_explore
        avg_explore_list.append(avg_step_explore)
        
        # Get average in all exploitation modes
        avg_step_exploit_no_rl = 0
        for trial in metrics_exploit_no_rl:
            avg_step_exploit_no_rl += trial['steps_in_trial']
        avg_step_exploit_no_rl /= NUMBER_OF_EXPLOIT_TRIALS_NO_RL
        avg_exploit_no_rl_list.append(avg_step_exploit_no_rl)

        avg_step_exploit_rl_start = 0
        for trial in metrics_exploit_rl_start:
            avg_step_exploit_rl_start += trial['steps_in_trial']
        avg_step_exploit_rl_start /= NUMBER_OF_EXPLOIT_TRIALS_RL_START
        avg_exploit_rl_start_list.append(avg_step_exploit_rl_start)

        avg_step_exploit_rl = 0
        for trial in metrics_exploit_rl:
            avg_step_exploit_rl += trial['steps_in_trial']
        avg_step_exploit_rl /= NUMBER_OF_EXPLOIT_TRIALS_RL
        avg_exploit_rl_list.append(avg_step_exploit_rl)
        
        # Get other metrics
        knowledge_list.append(metrics_exploit_rl[-1]['knowledge'])
        population_list.append(metrics_exploit_rl[-1]['population'])
        numerosity_list.append(metrics_exploit_rl[-1]['numerosity'])
        reliable_list.append(metrics_exploit_rl[-1]['reliable'])
        
        if i == n-1:
            print(env.id,'|','#' * (i+1), '|')
        else:
            print(env.id,'|','#' * (i+1) ,' ' * (n-i-2), '|')
    
    
    # Compute the means and standard deviations
    avg_explore = float(sum(avg_explore_list)) / n
    std_explore = statistics.pstdev(avg_explore_list)
    avg_exploit_no_rl = float(sum(avg_exploit_no_rl_list)) / n
    std_exploit_no_rl = statistics.pstdev(avg_exploit_no_rl_list)
    avg_exploit_rl_start = float(sum(avg_exploit_rl_start_list)) / n
    std_exploit_rl_start = statistics.pstdev(avg_exploit_rl_start_list)
    avg_exploit_rl = float(sum(avg_exploit_rl_list)) / n
    std_exploit_rl = statistics.pstdev(avg_exploit_rl_list)
    avg_knowledge = float(sum(knowledge_list)) / n
    std_knowledge = statistics.pstdev(knowledge_list)
    avg_population = float(sum(population_list)) / n
    std_population = statistics.pstdev(population_list)
    avg_numerosity = float(sum(numerosity_list)) / n
    std_numerosity = statistics.pstdev(numerosity_list)
    avg_reliable = float(sum(reliable_list)) / n
    std_reliable = statistics.pstdev(reliable_list)
    
    result = {
        'maze'             : env.id,
        'avg_explore'      : avg_explore,
        'std_explore'      : std_explore,
        'avg_exploit_no_rl'      : avg_exploit_no_rl,
        'std_exploit_no_rl'      : std_exploit_no_rl,
        'avg_exploit_rl_start'      : avg_exploit_rl_start,
        'std_exploit_rl_start'      : std_exploit_rl_start,
        'avg_exploit_rl'      : avg_exploit_rl,
        'std_exploit_rl'      : std_exploit_rl,
        'avg_knowledge'    : avg_knowledge,
        'std_knowledge'    : std_knowledge,
        'avg_population'   : avg_population,
        'std_population'   : std_population,
        'avg_numerosity'   : avg_numerosity,
        'std_numerosity'   : std_numerosity,
        'avg_reliable'     : avg_reliable,
        'std_reliable'     : std_reliable,
        'avg_exploit_no_rl_list' : avg_exploit_no_rl_list,
        'avg_exploit_rl_list' : avg_exploit_rl_list
    }
    
    print(result)
    
    return result

Custom functions for getting available environments in Gym depending on the type of mazes :

In [6]:
filter_envs_typeIII = lambda env: env.id.startswith("Maze10-") or env.id.startswith("MazeE1") \
    or env.id.startswith("MazeE2") or env.id.startswith("Woods10")

filter_envs_typeII = lambda env: env.id.startswith("MazeF4") or env.id.startswith("Maze7") \
    or env.id.startswith("MiyazakiB")

filter_envs_typeI = lambda env: env.id.startswith("MazeB") or env.id.startswith("MazeD") \
    or env.id.startswith("Littman") or env.id.startswith("MiyazakiA") \
    or env.id.startswith("Cassandra")

filter_envs_na = lambda env: env.id.startswith("MazeF1") or env.id.startswith("MazeF2") \
    or env.id.startswith("MazeF3") or env.id.startswith("Woods1-") \
    or env.id.startswith("Woods14") or env.id.startswith("Maze4") \
    or env.id.startswith("Maze5") or env.id.startswith("MazeA")

all_envs = [env for env in gym.envs.registry.all()]

Benchmarking without the genetic algorithms :

In [7]:
maze_envs = [env for env in all_envs if filter_envs_na(env) or filter_envs_typeI(env) or filter_envs_typeII(env) or filter_envs_typeIII(env)]

results = parmap.map(acs2_bench_on_maze, maze_envs, NUMBER_OF_ITERATIONS_TO_BENCH, pm_pbar=True, pm_processes=NB_OF_PROCESSES)

  0%|          | 0/24 [00:00<?, ?it/s]

Woods100-v0 | #                              |
MazeF1-v0 | #                              |
MazeF2-v0 | #                              |
Woods100-v0 | ##                             |
Woods1-v0 | #                              |
MazeF1-v0 | ##                             |
MazeF3-v0 | #                              |
Woods100-v0 | ###                            |
MazeF1-v0 | ###                            |
Woods100-v0 | ####                           |
MazeF2-v0 | ##                             |
Woods1-v0 | ##                             |
MazeF1-v0 | ####                           |
Woods101-v0 | #                              |
Littman57-v0 | #                              |
Cassandra4x4-v0 | #                              |
Woods100-v0 | #####                          |
MazeF1-v0 | #####                          |
MazeF3-v0 | ##                             |
Woods100-v0 | ######                         |
Woods14-v0 | #                              |
Maze7-v0 | #                   

  4%|▍         | 1/24 [04:02<1:32:46, 242.02s/it]

Littman57-v0 | ########                       |
Woods101-v0 | #######                        |
Woods101demi-v0 | ####                           |
MazeF2-v0 | ################               |
Maze10-v0 | ###                            |
MazeF1-v0 | ############################## |
{'maze': 'MazeF1-v0', 'avg_explore': 10.159999999999998, 'std_explore': 1.3692138863839598, 'avg_exploit_no_rl': 1.7921333333333334, 'std_exploit_no_rl': 0.03401150132267354, 'avg_exploit_rl_start': 1.815, 'std_exploit_rl_start': 0.0743303437365925, 'avg_exploit_rl': 1.7977999999999996, 'std_exploit_rl': 0.0313192166781568, 'avg_knowledge': 100.0, 'std_knowledge': 0.0, 'avg_population': 14.0, 'std_population': 0.0, 'avg_numerosity': 14.0, 'std_numerosity': 0.0, 'avg_reliable': 14.0, 'std_reliable': 0.0, 'avg_exploit_no_rl_list': [1.72, 1.786, 1.816, 1.774, 1.84, 1.832, 1.812, 1.826, 1.824, 1.85, 1.776, 1.75, 1.832, 1.76, 1.796, 1.8, 1.798, 1.744, 1.814, 1.788, 1.792, 1.754, 1.726, 1.832, 1.764, 1.762, 1.808, 1

  8%|▊         | 2/24 [04:10<1:02:59, 171.81s/it]

...

 92%|█████████▏| 22/24 [1:20:20<17:48, 534.16s/it]

MazeE1-v0 | ###########################    |
MazeE2-v0 | ##################             |
MazeE1-v0 | ############################   |
MazeE2-v0 | ###################            |
MazeE1-v0 | #############################  |
MazeE1-v0 | ############################## |
{'maze': 'MazeE1-v0', 'avg_explore': 20.938666666666663, 'std_explore': 2.8634661203203056, 'avg_exploit_no_rl': 37.84579999999999, 'std_exploit_no_rl': 6.132519424619325, 'avg_exploit_rl_start': 6.017666666666666, 'std_exploit_rl_start': 0.9847899381199131, 'avg_exploit_rl': 4.471933333333334, 'std_exploit_rl': 0.9666633310287277, 'avg_knowledge': 53.37499999999999, 'std_knowledge': 0.12499999999999929, 'avg_population': 349.8, 'std_population': 17.514565367145142, 'avg_numerosity': 349.8, 'std_numerosity': 17.514565367145142, 'avg_reliable': 157.06666666666666, 'std_reliable': 4.898525855352358, 'avg_exploit_no_rl_list': [28.116, 35.042, 38.442, 37.132, 39.384, 37.588, 33.442, 35.502, 36.274, 45.802, 41.91, 32.844, 38.

 96%|█████████▌| 23/24 [1:32:14<09:48, 588.13s/it]

MazeE2-v0 | ####################           |
MazeE2-v0 | #####################          |
MazeE2-v0 | ######################         |
MazeE2-v0 | #######################        |
MazeE2-v0 | ########################       |
MazeE2-v0 | #########################      |
MazeE2-v0 | ##########################     |
MazeE2-v0 | ###########################    |
MazeE2-v0 | ############################   |
MazeE2-v0 | #############################  |
MazeE2-v0 | ############################## |
{'maze': 'MazeE2-v0', 'avg_explore': 28.522666666666666, 'std_explore': 3.8281404827351304, 'avg_exploit_no_rl': 58.883133333333326, 'std_exploit_no_rl': 3.0389295344834095, 'avg_exploit_rl_start': 18.18933333333333, 'std_exploit_rl_start': 4.479970932445383, 'avg_exploit_rl': 36.50726666666666, 'std_exploit_rl': 3.7947058378160956, 'avg_knowledge': 36.875, 'std_knowledge': 2.0354905926848663, 'avg_population': 304.93333333333334, 'std_population': 23.878767881855953, 'avg_numerosity': 304.9333333333

100%|██████████| 24/24 [2:17:48<00:00, 344.52s/it] 


Parsing the previous result to get a markdown string for each environment :

In [8]:
markdown_str = ''

for item in results:
    markdown_str += '| ACS2 |'
    markdown_str += item['maze'] + '|'
    markdown_str += "{:.3f}".format(item['avg_explore']) + '|'
    markdown_str += "{:.3f}".format(item['std_explore']) + '|'
    markdown_str += "{:.3f}".format(item['avg_exploit_no_rl']) + '|'
    markdown_str += "{:.3f}".format(item['std_exploit_no_rl']) + '|'
    markdown_str += "{:.3f}".format(min(item['avg_exploit_no_rl_list'])) + '|'
    markdown_str += "{:.3f}".format(max(item['avg_exploit_no_rl_list'])) + '|'
    markdown_str += "{:.3f}".format(item['avg_exploit_rl']) + '|'
    markdown_str += "{:.3f}".format(item['std_exploit_rl']) + '|'
    markdown_str += "{:.3f}".format(min(item['avg_exploit_rl_list'])) + '|'
    markdown_str += "{:.3f}".format(max(item['avg_exploit_rl_list'])) + '|'
    markdown_str += "{:.3f}".format(item['avg_knowledge']) + '|'
    markdown_str += "{:.3f}".format(item['std_knowledge']) + '|'
    markdown_str += "{:.3f}".format(item['avg_population']) + '|'
    markdown_str += "{:.3f}".format(item['std_population']) + '|'
    markdown_str += "{:.3f}".format(item['avg_reliable']) + '|'
    markdown_str += "{:.3f}".format(item['std_reliable']) + '|'
    markdown_str += '\n'
    
print(markdown_str)

| ACS2 |Cassandra4x4-v0|13.626|1.452|2.953|0.423|2.212|4.038|2.915|0.415|2.204|3.952|50.741|0.489|130.700|10.799|54.467|3.222|
| ACS2 |Littman57-v0|22.339|3.969|37.166|27.421|3.590|70.384|4.319|1.165|3.562|8.300|73.496|3.256|34.633|0.657|27.200|1.013|
| ACS2 |Littman89-v0|32.953|5.725|34.542|27.867|4.558|63.952|5.361|0.782|4.584|7.624|71.429|0.000|100.300|3.917|71.167|0.522|
| ACS2 |MazeA-v0|49.525|4.249|4.242|0.086|4.098|4.512|4.230|0.088|4.070|4.514|100.000|0.000|102.633|2.810|102.633|2.810|
| ACS2 |MazeB-v0|34.960|4.203|10.676|13.965|3.874|54.072|4.507|0.425|4.002|5.420|80.425|0.176|160.600|8.689|118.867|3.314|
| ACS2 |MazeD-v0|25.211|2.664|2.775|0.058|2.656|2.902|2.784|0.061|2.658|2.920|87.535|0.187|128.667|5.723|107.167|3.670|
| ACS2 |MazeF4-v0|56.887|6.591|52.095|5.804|40.936|62.226|44.642|9.636|27.882|66.170|87.200|2.613|43.033|2.258|31.967|5.382|
| ACS2 |Maze4-v0|31.803|2.812|3.510|0.062|3.366|3.622|3.503|0.053|3.396|3.622|100.000|0.000|162.533|4.448|162.533|4.448|
| ACS2 |Maze

In [9]:
print(results)

[{'maze': 'Cassandra4x4-v0', 'avg_explore': 13.625999999999998, 'std_explore': 1.4515844217038616, 'avg_exploit_no_rl': 2.9532000000000007, 'std_exploit_no_rl': 0.4233654371029045, 'avg_exploit_rl_start': 2.988333333333333, 'std_exploit_rl_start': 0.407922242045657, 'avg_exploit_rl': 2.9152000000000005, 'std_exploit_rl': 0.4154623448641284, 'avg_knowledge': 50.740740740740755, 'std_knowledge': 0.4886560529645236, 'avg_population': 130.7, 'std_population': 10.798611021793498, 'avg_numerosity': 130.7, 'std_numerosity': 10.798611021793498, 'avg_reliable': 54.46666666666667, 'std_reliable': 3.2221455929585523, 'avg_exploit_no_rl_list': [2.842, 3.358, 2.28, 3.148, 3.676, 2.886, 2.782, 2.97, 2.922, 2.974, 2.892, 2.774, 3.144, 2.246, 2.696, 3.054, 2.944, 3.03, 2.992, 2.926, 2.932, 2.972, 3.462, 4.038, 2.388, 3.092, 3.78, 2.256, 2.212, 2.928], 'avg_exploit_rl_list': [2.702, 3.502, 2.204, 2.994, 3.42, 2.998, 2.83, 2.928, 2.96, 2.986, 2.944, 2.88, 3.14, 2.276, 2.63, 3.078, 2.922, 2.846, 3.04, 2.